In [ ]:
#install all the required dependancy libraries
!pip install torch

#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as torch
from torch.utils import data
from sklearn.preprocessing import StandardScaler
import io
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
%matplotlib inline


In [ ]:
btc = pd.read_csv('btc.csv')
btc.head()

,Date,Symbol,Open,High,Low,Close,Volume From,Volume To
0,5/26/2018,BTCUSD,7459.11,7640.46,7380.00,7520.00,2722.80,2.042265e+07
1,5/25/2018,BTCUSD,7584.15,7661.85,7326.94,7459.11,8491.93,6.342069e+07
2,5/24/2018,BTCUSD,7505.00,7734.99,7269.00,7584.15,11033.72,8.293137e+07
3,5/23/2018,BTCUSD,7987.70,8030.00,7433.19,7505.00,14905.99,1.148104e+08
4,5/22/2018,BTCUSD,8393.44,8400.00,7950.00,7987.70,6589.43,5.389753e+07


In [ ]:
class LoadData(data.Dataset):
    def __init__(self, dataset_dir, window_size=7, train=True):
        btc = pd.read_csv(dataset_dir)
        data_to_use=btc['Close'].values
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data_to_use.reshape(-1, 1))
        X, y = self.window_data(scaled_data, window_size)
        if train:
            self.data  = np.array(X[:1018]).astype(np.double)
            self.label = np.array(y[:1018]).astype(np.double)
            self.dataset_size = self.data.shape[0]
        else:
            self.data = np.array(X[1018:]).astype(np.double)
            self.label = np.array(y[1018:]).astype(np.double)
            self.dataset_size = self.data.shape[0]

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx] 

    def window_data(self, data, window_size):
        X = []
        y = []
    
        i = 0
        while (i + window_size) <= len(data) - 1:
            X.append(data[i:i+window_size])
            y.append(data[i+window_size])
            
            i += 1
        assert len(X) ==  len(y)
        return X, y

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size=1, hidden_feat=1, num_layers=100, seq_len=7, hidden_cell=None, output_size=1):
        super().__init__()
        self.hidden_layer_size = num_layers

        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_feat, num_layers=num_layers, batch_first=True)

        self.linear = torch.nn.Linear(seq_len, output_size)

        self.hidden_cell = hidden_cell

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        lstm_out = lstm_out.permute(0,2,1)
        predictions = self.linear(lstm_out)
        return predictions[:,:,0]

In [ ]:
dataset = LoadData('btc.csv', window_size=7)
dataset[0]

(array([[1.32443891],
        [1.30836052],
        [1.34137813],
        [1.32047807],
        [1.44793806],
        [1.55507627],
        [1.5876661 ]]), array([1.51339762]))

In [ ]:
def load_loaders(window_size):
    train_dataset = LoadData('btc.csv', window_size=window_size)
    train_loader = data.DataLoader(dataset=train_dataset, batch_size=w, drop_last=True)
    val_dataset = LoadData('btc.csv', window_size=window_size,train=False)
    val_loader = data.DataLoader(dataset=val_dataset, batch_size=1, drop_last=True)
    return train_loader, val_loader

In [ ]:
def load_optimizer(model,lr):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=lr)
    #optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    return optimizer

In [ ]:
def eval_val_loss(model, dataset, criterion, device):
    val_loss, n_batch = 0, 0
    model.eval()
    with torch.no_grad():
        for train_data in dataset:
            x, y = [x.to(device) for x in train_data]
            
            #x = x.permute(0, 2, 1)
            y_out= model(x)
            #y_out_unsqueeze = y_out[:,-1,-1].unsqueeze(-1)
            #value = criterion(y_out_unsqueeze, y)
            value = criterion(y_out, y)
            val_loss += value

            n_batch += 1
    
    return val_loss/n_batch

In [ ]:
def train(model, device, criterion, train_loader, optimizer, epoch, writer, params):
    # set model to training mode.
    steps = 0
    model.train()
    #h0 = torch.randn(params['window'], 7, params['hiden_layers']).to(device) # H da primeira célula (num_layers * num_directions, batch, hidden_size)
    #c0 = torch.randn(params['window'], 7, params['hiden_layers']).to(device) # C da primeira célula (num_layers * num_directions, batch, hidden_size)
    avg_loss = 0
    for train_data in train_loader:
        # move to GPU, if available
        x, y = [x.to(device) for x in train_data]
        # clear the gradients of all optimized variables.
        optimizer.zero_grad()

        # compute model output and loss.
        y_out = model(x.double()) #, (h0.double(),c0.double()))
        #import pdb;pdb.set_trace()
        #y_out_unsqueeze = y_out[:,-1,-1].unsqueeze(-1)
        #loss = criterion(y_out_unsqueeze, y)
        loss = criterion(y, y_out)
            
        # compute gradient of the loss with respect to model parameters.
        loss.backward()
        # performs updates using calculated gradients.
        optimizer.step()
        avg_loss += loss.item()

        steps += 1

    # write train loss on tensorboard.
    writer.add_scalars('loss_lr'+str(params['lr'])+'_hlayers_'+str(params['h']), {'window_'+str(params['window']):avg_loss/steps}, epoch+1)
    return avg_loss/steps


In [ ]:
def test(model, device, criterion, val_loader, epoch, writer, params):
    val_loss = eval_val_loss(model, val_loader, criterion, device)
    
    # write validation loss on tensorboard.
    writer.add_scalars('test_loss_lr'+str(params['lr'])+'_hlayers_'+str(params['h']), {'window_'+str(params['window']):val_loss}, epoch+1)
    return val_loss

In [ ]:
window_sizes = [7, 14]
lrs = [ 0.001, 0.01]
hiden_layers = [1,7,14]
criterion = torch.nn.MSELoss()
writer = SummaryWriter()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for w in window_sizes:
  for l in lrs:
    for h in hiden_layers:
        #print('w:'+str(w),'l:'+str(l),'h:'+str(h))
        hidden_cell = (torch.zeros(params['h'],params['window'],1).double().to(device),
                       torch.zeros(params['h'],params['window'],1).double().to(device))
        train_loader, val_loader = load_loaders(w)
        model = LSTM(input_size=1, hidden_feat=1, num_layers=h, hidden_cell=hidden_cell, seq_len=w).double().to(device)
        optimizer = load_optimizer(model,l)
        params = {}
        with tqdm(total=150) as t:
            for epoch in range(150):
                params['lr'] = l
                params['window'] = w
                params['h'] = h
                train_loss = train(model, device, criterion, train_loader, optimizer, epoch, writer, params)
                val_loss = test(model, device, criterion, val_loader, epoch, writer, params)
                loss_postfix = {'train_loss':'{:05.6f}'.format(train_loss),'val_loss':'{:05.6f}'.format(val_loss)}
                t.set_postfix(loss_postfix)
                t.update()
            
        print("===> Epochs Complete: Train Loss: {}. Val Loss {}".format(train_loss, val_loss), end=' ')
writer.close()


  0%|          | 0/150 [00:00<?, ?it/s]

===> Epochs Complete: Train Loss: 0.01289287882161587. Val Loss 1.5885606459320904e-05 

  0%|          | 0/150 [00:00<?, ?it/s]

===> Epochs Complete: Train Loss: 0.048742938269116. Val Loss 0.0001328748382501978 

  0%|          | 0/150 [00:00<?, ?it/s]

...

===> Epochs Complete: Train Loss: 0.030419445496868765. Val Loss 0.00022225715067018126 

  0%|          | 0/150 [00:00<?, ?it/s]

===> Epochs Complete: Train Loss: 1.208368985502319. Val Loss 0.3726153592074235 

100%|██████████| 150/150 [03:49<00:00,  1.53s/it, train_loss=1.208374, val_loss=0.372614]

===> Epochs Complete: Train Loss: 1.2083738318462616. Val Loss 0.37261439594031576 

In [ ]:
!zip -r /content/file.zip /content/runs
from google.colab import files
files.download("/content/file.zip")

  adding: content/runs/ (stored 0%)
  adding: content/runs/Mar31_10-41-45_8b696378d8c0/ (stored 0%)
  adding: content/runs/Mar31_10-41-45_8b696378d8c0/events.out.tfevents.1617187305.8b696378d8c0.66.4 (deflated 5%)
  adding: content/runs/Mar31_10-39-40_8b696378d8c0/ (stored 0%)
  adding: content/runs/Mar31_10-39-40_8b696378d8c0/events.out.tfevents.1617187180.8b696378d8c0.66.3 (deflated 5%)
  adding: content/runs/Mar31_10-57-51_8b696378d8c0/ (stored 0%)
  adding: content/runs/Mar31_10-57-51_8b696378d8c0/loss_lr0.001_hlayers_1_window_7/ (stored 0%)
  adding: content/runs/Mar31_10-57-51_8b696378d8c0/loss_lr0.001_hlayers_1_window_7/events.out.tfevents.1617188273.8b696378d8c0.66.13 (deflated 10%)
  adding: content/runs/Mar31_10-57-51_8b696378d8c0/test_loss_lr0.001_hlayers_1_window_7/ (stored 0%)
  adding: content/runs/Mar31_10-57-51_8b696378d8c0/test_loss_lr0.001_hlayers_1_window_7/events.out.tfevents.1617188273.8b696378d8c0.66.14 (deflated 10%)
  adding: content/runs/Mar31_10-57-51_8b696378

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>